# Getting started with data analysis using Pandas

Developed by Sam Maurer

Pandas is a popular Python libary for working with data: https://pandas.pydata.org

To use it in this notebook, we'll import it and give it the standard alias "pd":

In [ ]:
import pandas as pd

## 1. Pandas data objects: Series and DataFrames

Pandas has two standard structures for working with data: 
- "Series" are like lists
- "DataFrames" are like tables

Pandas data objects include some special metadata, like column names and indexes.

In [ ]:
x_list = [1.4, 5.98, 12, 0.0]

print(x_list)

In [ ]:
type(x_list)  # type() is a function that identifies the data type of a variable

In [ ]:
x_series = pd.Series({"x": [1.4, 5.98, 12, 0.0]})

print(x_series)

In [ ]:
type(x_series)

In [ ]:
data = pd.DataFrame({"x": [1.4, 5.98, 12, 0.0], "y": [17, 40, 52, 0]})

print(data)

In [ ]:
type(data)

As you might expect, each of the columns of a DataFrame is a Series:

In [ ]:
type(data['x'])

Pandas is part of the same ecosystem as Matplotlib and NumPy. Pandas DataFrames even provide built-in shortcuts to quickly plot data with Matplotlib.

This line allows the graphics to display directly in the notebook:

In [ ]:
%matplotlib inline

In [ ]:
data.plot.scatter('x', 'y')

## 2. Loading and displaying data

We're going to look at some U.S. housing cost data from Zillow: https://www.zillow.com/research/data/

Pandas can load data directly from a URL. In this case, we'll get the median asking rents for 2-bedroom units:

In [ ]:
url = "http://files.zillowstatic.com/research/public/Metro/Metro_MedianRentalPrice_2Bedroom.csv"

In [ ]:
df = pd.read_csv(url)

If that cell ran successfully, the data has been loaded.

Alternatively, you can load data "locally" -- from a file on the machine where Python is running. In this case, you'd pass to `pd.read_csv()` a **file path** instead of a URL.

And now we have a DataFrame!

In [ ]:
type(df)

How much data is there? `len()` is a standard Python function to get the length of things. If we pass it a DataFrame, it tells us the number of rows:

In [ ]:
len(df)

And there's a DataFrame property called `shape` that tells us a little more:

In [ ]:
df.shape  # rows x columns

(In case you're curious why there aren't any parentheses at the end of `df.shape`, it's because "shape" isn't a function that we're executing, but rather a metadata property of the DataFrame.)

We can display the data as a table, too, although Jupyter notebooks aren't great for browsing through raw data. Usually, displaying a few rows is enough to give us a sense of what we're working with:

In [ ]:
df.head()

But some of the columns are missing! We can fix this by adjusting a Pandas library setting:

In [ ]:
pd.set_option("display.max.columns", None)  # None means no maximum, a.k.a. everything

In [ ]:
df.head()

## 3. Cleaning up columns

It's pretty common that raw data files will have way more columns than you need, with obscure names as well.

My favorite approach is to start by creating _new_ columns with better names. This is more flexible than renaming columns in place because it's easier to undo.

In [ ]:
df['rent'] = df['2019-12']  # we only care about the most recent values
df['region'] = df['RegionName']
df['ranking'] = df['SizeRank']

What did this do?

In [ ]:
df.head(2)  # passing an integer gives us a custom number of rows

Then, we can make a copy of the DataFrame with just the columns we want:

In [ ]:
rents = df[['region', 'rent', 'ranking']].copy()

(For the curious: This will also work without `copy()` at the end. But sometimes you'll run into errors later on. **Without** `copy()`, the new variable only contains a reference to sections of the original DataFrame -- so if you try to change any of the data values, you run into problems. **With** copy, you truly duplicate the contents.)

In [ ]:
rents.head()

## 4. Descriptive statistics

Pandas provides a panel of pre-defined descriptive statistics that are a good place to start:

In [ ]:
rents.describe()

Too many decimals! We can fix this with another setting:

In [ ]:
pd.set_option("display.precision", 1)  # number of decimal places

In [ ]:
rents.describe()

The "descriptions" of non-numeric columns follows a different format, and we have to ask for them separately:

In [ ]:
rents['region'].describe()

There are also built-in functions to provide individual stats:

In [ ]:
rents['rent'].max()

You can use functions that are defined in other places, too:

In [ ]:
max(rents['rent'])

In [ ]:
import numpy as np

np.max(rents['rent'])

What's going on with the different kinds of syntax for calling functions?

Functions that you run by appending their name to the DataFrame, like `df['colname'].max()` are part of Pandas. (Technically, these are called "methods" rather than functions, for reasons we don't need to go into.) 

Here's the full list: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.html

With functions that come from other places, you pass the data you're evaluating into the function.

Here are some other common sources for math functions: 

- Core Python, like `max(list)`: https://docs.python.org/3/library/functions.html
- Core Python's math library: https://docs.python.org/3/library/math.html
- NumPy, like `np.max(list)`: https://numpy.readthedocs.io/en/latest/reference/routines.math.html


## 5. Digging into the data

Which cities have the highest and lowest rents?

In [ ]:
rents.nlargest(n=10, columns='rent')

In [ ]:
rents.nsmallest(n=10, columns='rent')

What if we want to limit it to the 50 largest metros? This requires filtering the data.

`df.loc[expression]` will filter a DataFrame to include only the specified rows and columns. For arcane reasons, the expression needs to refer to columns by their full identifier, not just their label.

Generally, we use square brackets when we're **accessing a subset of data**, or defining a list. We use parentheses everywhere else.

In [ ]:
rents.loc[rents['ranking'] < 50].nlargest(n=5, columns='rent')

In [ ]:
rents.loc[rents['ranking'] < 50].nsmallest(n=5, columns='rent')

Which are the large metros with rents over $1,800?

In [ ]:
rents.loc[(rents['ranking'] < 50) & (rents['rent'] > 1800)]

You need the extra parentheses so Python knows to evaluate each expression separately before calculating the union. (Otherwise there's a syntax error.) The symbol for "or" is `|`, from under the delete key.

If your expressions get complicated, you can break the lines between sets of parentheses:

In [ ]:
rents.loc[(rents['region'].str.contains('CA')) & 
          (rents['rent'] > 2000)]

## 6. Saving data tables to disk

Often you'll want to save the results of your analysis to disk, either for record keeping or so you can look at the data using other tools. This is easy:

In [ ]:
rents.to_csv('processed_rents.csv')

Now you should see it in the JupyterHub file browser.

## Exercises

Are there any cities in California, Oregon, or Washington where the mean rent is below $1500?

Can you plot a histogram of nationwide rents, using `.plot.hist()`? 

You'll need to run this on a single Series, rather than on a DataFrame as in the `.plot.scatter()` example. 

`.plot.hist()` does not require any arguments.

Can you make the histogram smoother? 

`.plot.hist()` takes an optional argument named `bins`, which defines how many buckets the data is divided into.

(You might want to create a new notebook for the next part, to keep things cleaner.)

Make a new variable showing the percentage change in rents from 2009 to 2019.

You can do this analogously to how we renamed the variables, but including a math expression on the right-hand side of the `=` assignment operator.

What's the mean and range of the changes?

Which cities had the largest and smallest changes?

What does a histogram of the changes look like? 